In [11]:
# -*- coding: utf-8 -*-
"""
@author: lrhao
@software: jupyter
@file: baseline.ipynb
@time: 2020-12-11
@description：
"""
import pandas 
import numpy 
import tqdm 
import os
import seaborn 
import math
import matplotlib
import sklearn
import time
import lightgbm
import random
import scorecardpy

print("pandas.Version:\t"+pd.__version__)
print("numpy.Version:\t"+np.__version__)
print("tqdm.Version:\t"+tqdm.__version__)
print("seaborn.Version:\t"+seaborn.__version__)
print("matplotlib.Version:\t"+matplotlib.__version__)
print("sklearn.Version:\t"+sklearn.__version__)
print("lightgbm.Version:\t"+lightgbm.__version__)
print("scorecardpy.Version:\t"+scorecardpy.__version__)


pandas.Version:	1.1.5
numpy.Version:	1.20.0
tqdm.Version:	4.26.0
seaborn.Version:	0.9.0
matplotlib.Version:	2.2.3
sklearn.Version:	0.19.2
lightgbm.Version:	2.1.1
scorecardpy.Version:	0.1.9.2


In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import math
import matplotlib.pyplot as plt
import warnings
import scorecardpy as sc
import time

In [2]:
train = pd.read_csv('../../../train.csv')
test = pd.read_csv('../../../test.csv')
submit = pd.read_csv('../../../submit.csv')
train.shape, test.shape, submit.shape

((40000, 21), (15000, 20), (15000, 2))

In [3]:
df = pd.concat([train, test], axis = 0).reset_index(drop = True)
#ZHIWU=1数据无意义，剔除该类数据
df = df[df['ZHIWU']!=1]
#剔除无效指标
del df['ZHIWU']
del df['XUELI']
del df['HYZK']
del df['ZHIYE']
del df['ZHICHEN']
del df['DWYJCE']

In [4]:
#对数值类的进行转化：
df['GRYJCE'] = df['GRYJCE'] - 237
df['GRJCJS'] = df['GRJCJS'] -237 
df['GRZHYE'] = df['GRZHYE'] - 237
df['GRZHSNJZYE'] = df ['GRZHSNJZYE'] -237
df['GRZHDNGJYE'] = df ['GRZHDNGJYE'] -237
df['DKFFE'] = df['DKFFE'] - 237
df['DKYE'] = df['DKYE']-237

In [5]:
cate_2_cols = ['XINGBIE']
cate_cols = [ 'DWJJLX', 'DWSSHY', 'GRZHZT']
# 记录需要做embedding的变量
cEmb = ['DWJJLX','DWSSHY']

In [6]:
num_cols = ['GRJCJS', 'GRZHYE', 'GRZHSNJZYE', 'GRZHDNGJYE', 'GRYJCE','DKFFE', 'DKYE', 'DKLL']

特征工程

In [7]:
df['missing_rate'] = (df.shape[1] - df.count(axis = 1)) / df.shape[1]

df['DKFFE_DKYE'] = df['DKFFE'] + df['DKYE']
df['DKFFE_DKY_multi_DKLL'] = (df['DKFFE'] + df['DKYE']) * df['DKLL']
df['GRZHDNGJYE_GRZHSNJZYE'] = df['GRZHDNGJYE'] + df['GRZHSNJZYE']
df['GRZHDNGJYE_GRZHSNJZYE1'] = df['GRZHDNGJYE'] - df['GRZHSNJZYE']
df['GRZHDNGJYE_GRZHSNJZYE2'] = df['GRZHDNGJYE']*df['GRZHDNGJYE'] + df['GRZHSNJZYE'] * df['GRZHSNJZYE']

df['DKFFE_multi_DKLL_ratio'] = df['DKFFE'] * df['DKLL'] / df['DKFFE_DKY_multi_DKLL']
df['DKYE_multi_DKLL_ratio'] = df['DKYE'] * df['DKLL'] / df['DKFFE_DKY_multi_DKLL']

df['GRZHYE_diff_GRZHDNGJYE'] = df['GRZHYE'] - df['GRZHDNGJYE']*12
df['GRZHYE_diff_GRZHSNJZYE'] = df['GRZHYE'] - df['GRZHSNJZYE']*1.3
df['GRZHYE_diff_GRZHDNGJYE1'] = df['GRZHYE'] - df['GRZHDNGJYE']
df['GRZHYE_diff_GRZHSNJZYE1'] = df['GRZHYE'] - df['GRZHSNJZYE']
#鬼知道，可能反应收入？
df['YEARPURINCM'] = df['GRZHYE']-df['GRZHSNJZYE']-df['GRZHDNGJYE']
#应还贷款月份，剩余需还贷款月份,已还贷款月份
df['HDZYF'] = df['DKFFE']*(df['DKLL']/100+1)/(df['GRYJCE']*2)
df['HDSYYF'] = df['DKYE']*(df['DKLL']/100+1)/(df['GRYJCE']*2)
df['HDYF'] = (df['DKFFE']-df['DKYE'])/(df['GRYJCE']*2)

#应还总额
df['HDZYFZE'] = df['DKFFE']*(df['DKLL']/100+1)
#待还总额
df['HDSYYFZE'] = df['DKYE']*(df['DKLL']/100+1)

#特征工程衍生的代码
# 图形旋转指标
df ['JS_YE'] = df['GRZHSNJZYE'] -20000* df['GRZHYE'].apply(lambda x: math.log(x+1))
df ['HDSYYF_HDZYF'] =df['HDSYYF'] -  df['HDZYF']*0.8
df ['GYE_GJC'] =df['GRZHYE'] -  (500-abs(df['GRYJCE']-1000))*100
df ['USAGE_RATE']=df['DKYE']/df['DKFFE']
df ['YE_USAGE'] = df['GRZHYE'] - df ['USAGE_RATE']*50000-50000
df ['YJCE2GRZHDNGJYE'] = df['GRYJCE']/ df ['GRZHDNGJYE']
df ['GRYJCE_RATE'] = df['GRYJCE']*df['GRYJCE']/2000/2000 + df ['USAGE_RATE']*df ['USAGE_RATE']
df ['GJJRATE'] = (df['GRYJCE'])/df['GRJCJS']                                                        # OKOKOK
df ['GJJRATE_GEN'] = df['GRJCJS'] - 60000*df['GJJRATE']                                             # OKOKOK
df ['GJJRATE_GEN3'] = df['GRJCJS'] - 200000*df['GJJRATE']                                           # OKOKOK
df ['GJJRATE_GEN2'] = df['GRYJCE'] - (600-df['GJJRATE']*2000)                                       # OKOKOK  
df ['DKFFE_DKYE251']= df['DKFFE'] - df['DKFFE']/df['DKYE']*251
df['GJJRATE_DIFF'] = df['GJJRATE'].apply(lambda x: 1 if x>0.1 else 0)

# 金额取整取余指标   # 这些指标暂时没有放到后面的指标衍生过程中！
df ['DKLT']= df['DKFFE']%100000
df ['DKZS']= df['DKFFE']//100000  
df ['YELT']= df['DKYE']%100000
df ['YEZS']= df['DKYE']//100000
df ['ZHLT']= df['GRZHYE']%10000
df ['ZHZS']= df['GRZHYE']//10000
df ['GJLT']= df['GRZHDNGJYE']%100000
df ['GJZS']= df['GRZHDNGJYE']//100000
df ['SNLT']= df['GRZHSNJZYE']%10000
df ['SNZS']= df['GRZHSNJZYE']//10000
df ['YJLT']= df['GRYJCE']%500
df ['YJZS']= df['GRYJCE']//500
df['SUBMONTH'] =  (df['GRZHYE']/df['GRYJCE']).apply(lambda x: math.log(x+1))                        # NEW NEW NEW
df['YJCE_LOG'] = (df['GRYJCE']).apply(lambda x: math.log(x+1))                                      # NEW NEW NEW
df['MONTH_LOG'] = df['SUBMONTH']-df['YJCE_LOG']                                                     # NEW NEW NEW

df['USAGEOVER'] = df['DKYE']==0 

gen_feats =  ['DKLT','YELT','ZHLT','GJLT','SNLT','YJLT','SUBMONTH','YJCE_LOG','MONTH_LOG',
              'GYE_GJC','USAGE_RATE','YE_USAGE','YJCE2GRZHDNGJYE',
           'GRYJCE_RATE','GJJRATE','GJJRATE_GEN','GJJRATE_GEN2','DKFFE_DKYE251',
'GRZHYE_diff_GRZHSNJZYE', 'GRZHYE_diff_GRZHDNGJYE1','GRZHYE_diff_GRZHSNJZYE1', 'YEARPURINCM','HDZYF',
            'HDSYYF','HDYF','HDZYFZE','HDSYYFZE','JS_YE','HDSYYF_HDZYF',
'DKFFE_DKYE', 'DKFFE_DKY_multi_DKLL','GRZHDNGJYE_GRZHSNJZYE','GRZHDNGJYE_GRZHSNJZYE1',  'GRZHDNGJYE_GRZHSNJZYE2',
             'DKFFE_multi_DKLL_ratio', 'DKYE_multi_DKLL_ratio', 'GRZHYE_diff_GRZHDNGJYE']
            
cate_cols += ['DKZS','YEZS','ZHZS','GJZS','SNZS','YJZS','USAGEOVER']

In [8]:
#read in the feature importance data
validIndics = list()
validIndicsAll = list()
level = 30
for i in range(1,5):
    dimp = pd.read_csv(f"FImp{i}.csv")
    validIndics.append(dimp[dimp['importance']>level]['column'].values.tolist())
    validIndicsAll +=  dimp[dimp['importance']>level]['column'].values.tolist()
print([len(i) for i in validIndics])

[898, 919, 778, 845]


In [9]:
import time
def get_age(df,col = 'age'):
    df[col+"_genFeat1"]=(df['age'] > 18).astype(int)
    df[col+"_genFeat2"]=(df['age'] > 25).astype(int)
    df[col+"_genFeat3"]=(df['age'] > 30).astype(int)
    df[col+"_genFeat4"]=(df['age'] > 35).astype(int)
    df[col+"_genFeat5"]=(df['age'] > 40).astype(int)
    df[col+"_genFeat6"]=(df['age'] > 45).astype(int)
    return df, [col + f'_genFeat{i}' for i in range(1, 7)]

# BORN MONTH
tNow = time.mktime(time.strptime( '2021-01-12 00:00:00','%Y-%m-%d %H:%M:%S'))
df['bornmonth'] = df['CSNY'].apply(lambda x: time.localtime(x)[1])

#这个怎么换算出来的啊，超哥
df['age'] = ((1609430399 - df['CSNY']) / (365 * 24 * 3600)).astype(int)
df, genFeats1 = get_age(df, col = 'age')

#sns.distplot(df['age'][df['age'] > 0])

In [10]:
def get_daikuanYE(df,col):
    df[col + '_genFeat1'] = (df[col] > 100000).astype(int)
    df[col + '_genFeat2'] = (df[col] > 120000).astype(int)
    df[col + '_genFeat3'] = (df[col] > 140000).astype(int)
    df[col + '_genFeat4'] = (df[col] > 180000).astype(int)
    df[col + '_genFeat5'] = (df[col] > 220000).astype(int)
    df[col + '_genFeat6'] = (df[col] > 260000).astype(int)
    df[col + '_genFeat7'] = (df[col] > 300000).astype(int)
    return df, [col + f'_genFeat{i}' for i in range(1, 8)]

df, genFeats2 = get_daikuanYE(df, col = 'DKYE')
df, genFeats3 = get_daikuanYE(df, col = 'DKFFE')

In [11]:
from tqdm import tqdm

In [12]:
# One-hot 编码
for f in tqdm(cate_cols):
    df[f] = df[f].map(dict(zip(df[f].unique(), range(df[f].nunique()))))
    df[f + '_count'] = df[f].map(df[f].value_counts())
    df = pd.concat([df,pd.get_dummies(df[f],prefix=f"{f}")],axis=1)

#离散型特征关联
cate_cols_combine = [[cate_cols[i], cate_cols[j]] for i in range(len(cate_cols)) \
                     for j in range(i + 1, len(cate_cols))]

#离散型联合计数占比
for f1, f2 in tqdm(cate_cols_combine):
    df['{}_{}_count'.format(f1, f2)] = df.groupby([f1, f2])['id'].transform('count')
    df['{}_in_{}_prop'.format(f1, f2)] = df['{}_{}_count'.format(f1, f2)] / df[f2 + '_count']
    df['{}_in_{}_prop'.format(f2, f1)] = df['{}_{}_count'.format(f1, f2)] / df[f1 + '_count']


#离散型变量下各个数值型指标统计
for f1 in tqdm(cate_cols):
    g = df.groupby(f1)
    for f2 in num_cols + gen_feats:
        for stat in ['sum', 'mean', 'std', 'max', 'min', 'std']:
            if '{}_{}_{}'.format(f1, f2, stat)  in validIndicsAll:
                df['{}_{}_{}'.format(f1, f2, stat)] = g[f2].transform(stat)
    for f3 in genFeats2 + genFeats3:
        for stat in ['sum', 'mean']:
            if '{}_{}_{}'.format(f1, f3, stat) in validIndicsAll:
                df['{}_{}_{}'.format(f1, f3, stat)] = g[f3].transform(stat)

    
#离散型变量下各个数值特征指标
num_cols_gen_feats = num_cols + gen_feats
for f1 in tqdm(num_cols_gen_feats):
    g = df.groupby(f1)
    for f2 in num_cols_gen_feats:
        if f1 != f2:
            for stat in ['sum', 'mean', 'std', 'max', 'min', 'std']:
                if '{}_{}_{}'.format(f1, f2, stat) in validIndicsAll:
                    df['{}_{}_{}'.format(f1, f2, stat)] = g[f2].transform(stat)

#连续性数值间的特征工程
for i in tqdm(range(len(num_cols_gen_feats))):
    for j in range(i + 1, len(num_cols_gen_feats)):
        if f'numsOf_{num_cols_gen_feats[i]}_{num_cols_gen_feats[j]}_add'  in validIndicsAll:
            df[f'numsOf_{num_cols_gen_feats[i]}_{num_cols_gen_feats[j]}_add'] = df[num_cols_gen_feats[i]] + df[num_cols_gen_feats[j]]
        if f'numsOf_{num_cols_gen_feats[i]}_{num_cols_gen_feats[j]}_diff'  in validIndicsAll:
            df[f'numsOf_{num_cols_gen_feats[i]}_{num_cols_gen_feats[j]}_diff'] = df[num_cols_gen_feats[i]] - df[num_cols_gen_feats[j]]
        if f'numsOf_{num_cols_gen_feats[i]}_{num_cols_gen_feats[j]}_multi'  in validIndicsAll:
            df[f'numsOf_{num_cols_gen_feats[i]}_{num_cols_gen_feats[j]}_multi'] = df[num_cols_gen_feats[i]] * df[num_cols_gen_feats[j]]
        if f'numsOf_{num_cols_gen_feats[i]}_{num_cols_gen_feats[j]}_div'  in validIndicsAll:
            df[f'numsOf_{num_cols_gen_feats[i]}_{num_cols_gen_feats[j]}_div'] = df[num_cols_gen_feats[i]] / (df[num_cols_gen_feats[j]] + 0.0000000001)

100%|██████████| 45/45 [00:27<00:00,  1.61it/s]


训练集、测试集

In [13]:
train_df = df[df['label'].isna() == False].reset_index(drop=True)
test_df = df[df['label'].isna() == True].reset_index(drop=True)
display(train_df.shape, test_df.shape)

(39994, 2919)

(10000, 2919)

In [14]:
drop_feats = []
drop_feats = [f for f in train_df.columns if train_df[f].nunique() == 1 or train_df[f].nunique() == 0]
len(drop_feats), drop_feats

(4, ['missing_rate', 'ZHZS_31', 'SNZS_30', 'SNZS_31'])

In [15]:
def tpr_weight_funtion(y_true,y_predict):
    d = pd.DataFrame()
    d['prob'] = list(y_predict)
    d['y'] = list(y_true)
    d = d.sort_values(['prob'], ascending=[0])
    y = d.y
    PosAll = pd.Series(y).value_counts()[1]
    NegAll = pd.Series(y).value_counts()[0]
    pCumsum = d['y'].cumsum()
    nCumsum = np.arange(len(y)) - pCumsum + 1
    pCumsumPer = pCumsum / PosAll
    nCumsumPer = nCumsum / NegAll
    TR1 = pCumsumPer[abs(nCumsumPer-0.001).idxmin()]
    TR2 = pCumsumPer[abs(nCumsumPer-0.005).idxmin()]
    TR3 = pCumsumPer[abs(nCumsumPer-0.01).idxmin()]
    
    return ('TPR',0.4 * TR1 + 0.3 * TR2 + 0.3 * TR3,True)

In [16]:
from sklearn.model_selection import StratifiedKFold
from lightgbm.sklearn import LGBMClassifier
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
import time
from lightgbm.callback import reset_parameter
import math

In [17]:
import random
from sklearn.metrics import balanced_accuracy_score

In [18]:
oof = np.zeros((len(validIndics),train_df.shape[0]))
middleresult =  np.zeros((len(validIndics),test_df.shape[0]))
seeds = [1023]
for imodel in range(len(validIndics)):

    clf = LGBMClassifier(
    boosting_type = 'goss',
    learning_rate=0.03,
    n_estimators=4023,
    num_leaves=101,  # 只进行两层叶子节点的交叉
    random_state=1023,
    metric=None,
    is_unbalance=True,
    reg_alpha=0.005,
    reg_lambda=0.005,
    min_data_in_leaf = 5,
    )
    cols = list(set([col for col in train_df.columns if col not in ['id', 'label'] + drop_feats]).intersection(set(validIndics[imodel])))

    for seed in seeds:
        print(len(cols),seed)
        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
        for i, (trn_idx, val_idx) in enumerate(skf.split(train_df, train_df['label'])):
            trn_x, trn_y = train_df[cols].iloc[trn_idx].reset_index(drop=True), train_df['label'].values[trn_idx]
            val_x, val_y = train_df[cols].iloc[val_idx].reset_index(drop=True), train_df['label'].values[val_idx]
            clf.fit(
                        trn_x, trn_y,
                        eval_set=[(val_x, val_y)],
                        categorical_feature=list(set(cate_cols).intersection(cols)),
                        eval_metric='binary_error',
                       early_stopping_rounds=200,
                       verbose=200
                    )
            oof[imodel,val_idx] = clf.predict_proba(val_x)[:, 1]
            middleresult[imodel] += clf.predict_proba(test_df[cols])[:, 1] / skf.n_splits / len(seeds)     
        tpr = round(tpr_weight_funtion(train_df['label'],oof[imodel])[1], 6)
        print("======== model"+str(imodel)+' TPR:'+str(tpr))

898 1023


/home/ubuntu/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1038: UserWarning: categorical_feature in Dataset is overrided. New categorical_feature is ['DWJJLX', 'DWSSHY']
  warnings.warn('categorical_feature in Dataset is overrided. New categorical_feature is {}'.format(sorted(list(categorical_feature))))


Training until validation scores don't improve for 200 rounds.
[200]	valid_0's binary_error: 0.0427553
[400]	valid_0's binary_error: 0.0336292
[600]	valid_0's binary_error: 0.0325041
Early stopping, best iteration is:
[567]	valid_0's binary_error: 0.032379
Training until validation scores don't improve for 200 rounds.
[200]	valid_0's binary_error: 0.0405051
[400]	valid_0's binary_error: 0.0336292
[600]	valid_0's binary_error: 0.031629
[800]	valid_0's binary_error: 0.031629
Early stopping, best iteration is:
[626]	valid_0's binary_error: 0.0312539
Training until validation scores don't improve for 200 rounds.
[200]	valid_0's binary_error: 0.0452557
[400]	valid_0's binary_error: 0.0382548
[600]	valid_0's binary_error: 0.0356295
[800]	valid_0's binary_error: 0.0342543
[1000]	valid_0's binary_error: 0.0332542
Early stopping, best iteration is:
[968]	valid_0's binary_error: 0.0330041
Training until validation scores don't improve for 200 rounds.
[200]	valid_0's binary_error: 0.0471309
[400]

/home/ubuntu/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1038: UserWarning: categorical_feature in Dataset is overrided. New categorical_feature is ['DWJJLX', 'DWSSHY']
  warnings.warn('categorical_feature in Dataset is overrided. New categorical_feature is {}'.format(sorted(list(categorical_feature))))


Training until validation scores don't improve for 200 rounds.
[200]	valid_0's binary_error: 0.0415052
[400]	valid_0's binary_error: 0.0338792
[600]	valid_0's binary_error: 0.032379
[800]	valid_0's binary_error: 0.031754
[1000]	valid_0's binary_error: 0.0313789
Early stopping, best iteration is:
[902]	valid_0's binary_error: 0.0312539
Training until validation scores don't improve for 200 rounds.
[200]	valid_0's binary_error: 0.0386298
[400]	valid_0's binary_error: 0.031879
[600]	valid_0's binary_error: 0.0300038
Early stopping, best iteration is:
[518]	valid_0's binary_error: 0.0300038
Training until validation scores don't improve for 200 rounds.
[200]	valid_0's binary_error: 0.0452557
[400]	valid_0's binary_error: 0.0358795
[600]	valid_0's binary_error: 0.0331291
[800]	valid_0's binary_error: 0.0325041
[1000]	valid_0's binary_error: 0.0325041
Early stopping, best iteration is:
[844]	valid_0's binary_error: 0.032004
Training until validation scores don't improve for 200 rounds.
[200]

/home/ubuntu/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1038: UserWarning: categorical_feature in Dataset is overrided. New categorical_feature is ['DWJJLX', 'DWSSHY']
  warnings.warn('categorical_feature in Dataset is overrided. New categorical_feature is {}'.format(sorted(list(categorical_feature))))


Training until validation scores don't improve for 200 rounds.
[200]	valid_0's binary_error: 0.0415052
[400]	valid_0's binary_error: 0.0332542
[600]	valid_0's binary_error: 0.0315039
[800]	valid_0's binary_error: 0.0310039
Early stopping, best iteration is:
[653]	valid_0's binary_error: 0.0308789
Training until validation scores don't improve for 200 rounds.
[200]	valid_0's binary_error: 0.03988
[400]	valid_0's binary_error: 0.0341293
[600]	valid_0's binary_error: 0.0313789
[800]	valid_0's binary_error: 0.0301288
[1000]	valid_0's binary_error: 0.0296287
[1200]	valid_0's binary_error: 0.0295037
Early stopping, best iteration is:
[1015]	valid_0's binary_error: 0.0292537
Training until validation scores don't improve for 200 rounds.
[200]	valid_0's binary_error: 0.0442555
[400]	valid_0's binary_error: 0.0348794
[600]	valid_0's binary_error: 0.0331291
[800]	valid_0's binary_error: 0.0330041
Early stopping, best iteration is:
[669]	valid_0's binary_error: 0.032379
Training until validation 

/home/ubuntu/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1038: UserWarning: categorical_feature in Dataset is overrided. New categorical_feature is ['DWJJLX', 'DWSSHY']
  warnings.warn('categorical_feature in Dataset is overrided. New categorical_feature is {}'.format(sorted(list(categorical_feature))))


Training until validation scores don't improve for 200 rounds.
[200]	valid_0's binary_error: 0.0422553
[400]	valid_0's binary_error: 0.0341293
[600]	valid_0's binary_error: 0.0332542
[800]	valid_0's binary_error: 0.0325041
Early stopping, best iteration is:
[761]	valid_0's binary_error: 0.032129
Training until validation scores don't improve for 200 rounds.
[200]	valid_0's binary_error: 0.0390049
[400]	valid_0's binary_error: 0.0328791
[600]	valid_0's binary_error: 0.0313789
Early stopping, best iteration is:
[578]	valid_0's binary_error: 0.0311289
Training until validation scores don't improve for 200 rounds.
[200]	valid_0's binary_error: 0.0455057
[400]	valid_0's binary_error: 0.0357545
[600]	valid_0's binary_error: 0.0336292
[800]	valid_0's binary_error: 0.032004
[1000]	valid_0's binary_error: 0.031629
[1200]	valid_0's binary_error: 0.031879
Early stopping, best iteration is:
[1038]	valid_0's binary_error: 0.0313789
Training until validation scores don't improve for 200 rounds.
[200

KeyboardInterrupt: 

In [19]:
stackDF_train = pd.DataFrame(oof.T)
stackDF_train['label'] = train_df['label']
stackDF_test = pd.DataFrame(middleresult.T)

In [20]:
for i in stackDF_train.columns:
    print(round(tpr_weight_funtion(train_df['label'],stackDF_train[i])[1], 6))

0.562423
0.556075
0.561589
0.556692
1.0


In [163]:
cols = list(set(stackDF_train.columns)-set(['label']))
from sklearn.linear_model import LogisticRegressionCV,LinearRegression
import sklearn
from sklearn.model_selection import train_test_split

for i in sorted(sklearn.metrics.SCORERS.keys()):
    try:
        X_train,X_test,Y_train,Y_test = train_test_split(stackDF_train[cols],stackDF_train['label'],test_size=0.1,random_state=0)
        lr = LogisticRegressionCV(fit_intercept=True,Cs=np.logspace(-2,2,20),cv=2,penalty="l2",solver="lbfgs",tol=0.01,
                                  class_weight={0:0.1,1:1},scoring=i)
        re = lr.fit(X_train,Y_train)

        r = re.score(X_train,Y_train)
        P_test = [i[1] for i in re.predict_proba(X_test)]
        print(i+str(tpr_weight_funtion(Y_test,P_test)))
    except Exception:
        continue

accuracy('TPR', 0.5170648464163823, True)
adjusted_mutual_info_score('TPR', 0.5170648464163823, True)
adjusted_rand_score('TPR', 0.5170648464163823, True)
average_precision('TPR', 0.5102389078498294, True)
balanced_accuracy('TPR', 0.5102389078498294, True)
completeness_score('TPR', 0.5170648464163823, True)
explained_variance('TPR', 0.5170648464163823, True)
f1('TPR', 0.5170648464163823, True)
f1_macro('TPR', 0.5170648464163823, True)
f1_micro('TPR', 0.5170648464163823, True)
f1_weighted('TPR', 0.5170648464163823, True)
fowlkes_mallows_score('TPR', 0.5170648464163823, True)
homogeneity_score('TPR', 0.5136518771331058, True)
jaccard('TPR', 0.5170648464163823, True)
jaccard_macro('TPR', 0.5170648464163823, True)
jaccard_micro('TPR', 0.5170648464163823, True)
jaccard_weighted('TPR', 0.5170648464163823, True)
max_error('TPR', 0.5170648464163823, True)
mutual_info_score('TPR', 0.5136518771331058, True)
neg_brier_score('TPR', 0.5102389078498294, True)
neg_log_loss('TPR', 0.5180887372013652, 

In [149]:
import sklearn
sorted(sklearn.metrics.SCORERS.keys())

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'roc_auc_ovo',
 'roc_auc_ovo_weighted',
 'roc_auc_ovr',
 'roc_auc_ovr_weighted',
 'v_measure_score']

In [143]:
help(LogisticRegressionCV)

Help on class LogisticRegressionCV in module sklearn.linear_model._logistic:

class LogisticRegressionCV(LogisticRegression, sklearn.base.BaseEstimator, sklearn.linear_model._base.LinearClassifierMixin)
 |  Logistic Regression CV (aka logit, MaxEnt) classifier.
 |  
 |  See glossary entry for :term:`cross-validation estimator`.
 |  
 |  This class implements logistic regression using liblinear, newton-cg, sag
 |  of lbfgs optimizer. The newton-cg, sag and lbfgs solvers support only L2
 |  regularization with primal formulation. The liblinear solver supports both
 |  L1 and L2 regularization, with a dual formulation only for the L2 penalty.
 |  Elastic-Net penalty is only supported by the saga solver.
 |  
 |  For the grid of `Cs` values and `l1_ratios` values, the best hyperparameter
 |  is selected by the cross-validator
 |  :class:`~sklearn.model_selection.StratifiedKFold`, but it can be changed
 |  using the :term:`cv` parameter. The 'newton-cg', 'sag', 'saga' and 'lbfgs'
 |  solver

In [129]:
clf.fit(
    stackDF_train[cols],stackDF_train['label'],
            eval_metric='auc',
           verbose=200,
        )
test_df['prob'] = clf.predict_proba(stackDF_test[cols])[:, 1]

In [21]:
#指标重要性分析
dImp = pd.DataFrame({
        'column': cols,
        'importance': [sum(i)/len(i) for i in np.array([ i.feature_importances_ for i in modellist ]).transpose()] ,
    }).sort_values(by='importance',ascending=False).reset_index()
print(len(dImp[dImp['importance']<1]['column'].values.tolist()))

1411


In [23]:
#dImp.to_csv('FImp4.csv')

In [130]:
submit['id'] = test_df['id']
submit['label'] = test_df['prob']
submit = pd.merge(test,submit,on='id',how='left')[['id','label']]
submit['label'] = submit['label'].apply(lambda x: 0 if pd.isnull(x) else x)


submit.to_csv('../sub/submission{}_{}.csv'.format(tpr, round(np.mean(val_aucs), 6)), index = False)
submit.shape


/home/ubuntu/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/ubuntu/.local/lib/python3.6/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


(15000, 2)